# Análise de sequências de peptídeos para uso antiviral

In [1]:
# Instalação de bibliotecas necessárias

! pip3 install pandas
! pip3 install matplotlib
! pip3 install numpy
! pip3 install sklearn
! pip3 install torch
! pip3 install blosum

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no lon

In [2]:
# Importação de bibliotecas de uso geral

import pandas as pd
import sklearn
import statistics 
import matplotlib.pyplot as plt
from decimal import Decimal
import numpy as np

In [3]:
# Leitura do conjunto de dados

data = pd.read_csv('AVPdb.csv',sep=';')
display(data.head())

,Id,Sequence,Length,Virus,Abbreviation,Family,Nomenclature,Source,Uniprot,Cell_Line,Inhibition/IC50,Unit,Target,Assay,Accession,Title,Authors,Reference
0,AVP0001,PYVGSGLYRR,10,Avian myeloblastosis virus,AMV,Retroviridae,p2-p10,RSV gag polyprotein,P03354,-,50,μM,Release,Protease activity assay,1331099,Mechanism of inhibition of the retroviral prot...,"Cameron CE, Grinde B, Jentoft J, Leis J, Weber...",J Biol Chem. 1992
1,AVP0002,SMIENLEYM,9,Lymphocytic choriomeningitis virus,LCMV,Arenaviridae,-,Synthetic,NaN,MC57,50,μM,CTL lysis,In vitro cytotoxicity assay,1383569,Design of high-affinity major histocompatibili...,"Gairin JE, Oldstone MB.",J Virol. 1992
2,AVP0003,ECRSTSYAGAVVNDL,15,Herpes simplex virus 1,HSV 1,Herpesviridae,H2-(1-15),HSV ribonucleotide reductase subunit 2,P10224,BHK-21/C,42,μM,Replication,Ribonucleotide reductase assay,3040743,Structure-activity studies on synthetic peptid...,"Gaudreau P, Michaud J, Cohen EA, Langelier Y, ...",J Biol Chem. 1987
3,AVP0004,STSYAGAVVNDL,12,Herpes simplex virus 1,HSV 1,Herpesviridae,H2-(4-15),HSV ribonucleotide reductase subunit 2,P10224,BHK-21/C,29,μM,Replication,Ribonucleotide reductase assay,3040743,Structure-activity studies on synthetic peptid...,"Gaudreau P, Michaud J, Cohen EA, Langelier Y, ...",J Biol Chem. 1987
4,AVP0005,YAGAVVNDL,9,Herpes simplex virus 1,HSV 1,Herpesviridae,H2-(7-15),HSV ribonucleotide reductase subunit 2,P10224,BHK-21/C,60,μM,Replication,Ribonucleotide reductase assay,3040743,Structure-activity studies on synthetic peptid...,"Gaudreau P, Michaud J, Cohen EA, Langelier Y, ...",J Biol Chem. 1987


In [4]:
# Funções auxiliares para o processo

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Validação cruzada em 10-folds que será utilizada em todo o trabalho
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

def get_subsequences(input_sequence, size):
    sequence = input_sequence.replace(' ', '')
    subsequences = []
    if len(sequence) < size:
        seq = sequence
        while(len(seq) < size):
            seq = seq + '0'
        subsequences.append(seq)
        return subsequences
    n_subsequences = len(sequence) - size + 1
    for i in range(n_subsequences):
        subsequences.append(sequence[i:i+size])
    return subsequences

def extract_aminoacid_symbols(data):
    symbols = ['0']
    for index, x in data.iterrows():
        x_sequence = x['Sequence'].replace(' ', '')
        for x_symbol in x_sequence:
            present = False
            for symbol in symbols:
                if x_symbol == symbol:
                    present = True
                    break
            if not present:
                symbols.append(x_symbol)
    encoder = LabelEncoder()
    encoder.fit(symbols)
    return symbols, encoder

def build_basic_dataframe(data, size):
    subsequences = []
    virus_ids = []
    virus_abbreviations = []
    sars_cov = []
    hcv = []
    for index, x in data.iterrows():
        abbreviation = x['Abbreviation']
        x_subsequences = get_subsequences(x['Sequence'], size)
        for x_subsequence in x_subsequences:
            subsequences.append(x_subsequence)
            virus_abbreviations.append(abbreviation)
            hcv.append(1 if abbreviation == 'HCV' else 0)
            sars_cov.append(1 if abbreviation == 'SARS-CoV' else 0)
    encoder = LabelEncoder()
    encoder.fit(virus_abbreviations)
    virus_ids = encoder.transform(virus_abbreviations)
    return pd.DataFrame(list(zip(subsequences, virus_ids, hcv, sars_cov)), columns =['subsequence', 'virus_id', 'hcv', 'sars_cov']), encoder

def build_and_print_decision_tree(X, y, replaces):
    max_depth = 8
    clf = DecisionTreeClassifier(random_state=0, max_depth=max_depth)
    print('Decision Tree:', statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1)))
    clf.fit(X, y)
    tree_text = export_text(clf, feature_names=X.columns.tolist(), max_depth=max_depth)
    tree_text = tree_text.replace('class: 0', 'No SARS-CoV inhibition observed')
    tree_text = tree_text.replace('class: 1', 'Likely SARS-CoV inhibition observed')
    for r in replaces:
        tree_text = tree_text.replace(r[0], r[1])
    print(tree_text)

def benchmark_classifiers(X, y):
    clf = GaussianNB()
    print('Gaussian Naive Bayes: ', statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1)))

    clf = KNeighborsClassifier()
    print('KNN: ', statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1)))
    
    clf = SVC(random_state=0, kernel='poly', C=1, degree=2)
    print('Polynomial SVM degree 2: ', statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1)))
    
    clf = RandomForestClassifier(random_state=0)
    print('Random Forest: ', statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1)))

    clf = DecisionTreeClassifier(random_state=0)
    print('Decision Tree:', statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1)))

## Modelo de tradução posicional

Neste modelo cada posição de uma subsequência será modelada como uma feature e indicaremos qual aminoácido está presente em uma dada posição.

In [5]:
def build_encoder_translation_dataframe(data, size):
    df, virus_encoder = build_basic_dataframe(data, size)
    _, symbol_encoder = extract_aminoacid_symbols(data)
    translations = []
    for index, x in df.iterrows():
        subsequence = x['subsequence']
        subsequence_arr = []
        for s in subsequence:
           subsequence_arr.append(s) 
        encoded = symbol_encoder.transform(subsequence_arr)
        dictionary = {
            'virus_id': x['virus_id'],
            'sars_cov': x['sars_cov'],
            'hcv': x['hcv']
        }
        for i in range(size):
            dictionary['pos_'+str(i+1)] = encoded[i]
        translations.append(dictionary)
    return pd.DataFrame(translations), virus_encoder, symbol_encoder

In [6]:
df, virus_encoder, symbol_encoder = build_encoder_translation_dataframe(data, 8)
df.head()

,virus_id,sars_cov,hcv,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8
0,1,0,0,13,20,18,6,16,6,10,20
1,1,0,0,20,18,6,16,6,10,20,15
2,1,0,0,18,6,16,6,10,20,15,15
3,41,0,0,16,11,8,4,12,10,4,20
4,41,0,0,11,8,4,12,10,4,20,11


In [7]:
X = df.drop(columns=['virus_id', 'sars_cov', 'hcv'])

In [8]:
y = df['virus_id'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.1896912941829309
KNN:  0.5715763999617016
Polynomial SVM degree 2:  0.24315763169619237
Random Forest:  0.7178487168975394
Decision Tree: 0.6751602859105209


In [9]:
y = df['sars_cov'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.8998036165967541
KNN:  0.9609683020883633
Polynomial SVM degree 2:  0.8998036165967541
Random Forest:  0.980060800301645
Decision Tree: 0.9606603136265182


In [10]:
y = df['hcv'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.8501482731747979
KNN:  0.909311137414136
Polynomial SVM degree 2:  0.8501482731747979
Random Forest:  0.9693983227523437
Decision Tree: 0.9404902559742796


## Modelo de posição dos aminoácidos

Neste modelo teremos `tamanho da subsequência` * `número de aminoácidos` features e cada uma delas indicará se um aminoácido está presente em uma dada posição, o modelo então será uma matriz bastante esparsa contendo apenas valores 0 ou 1.

In [11]:
def build_aminoacid_position_dataframe(data, size):
    df, virus_encoder = build_basic_dataframe(data, size)
    symbols, _ = extract_aminoacid_symbols(data)
    aminoacid_positions = []
    for index, x in df.iterrows():
        subsequence = x['subsequence']
        dictionary = {
            'virus_id': x['virus_id'],
            'sars_cov': x['sars_cov'],
            'hcv': x['hcv']
        }
        for i in range(len(subsequence)):
            if subsequence[i] != '0':
                dictionary[subsequence[i] + '_' + str(i+1)] = 1
            for s in symbols:
                if s != subsequence[i] and s != '0':
                    dictionary[s + '_' + str(i+1)] = 0
        aminoacid_positions.append(dictionary)
    return pd.DataFrame(aminoacid_positions), virus_encoder

In [12]:
df, virus_encoder = build_aminoacid_position_dataframe(data, 8)
df.head()

,virus_id,sars_cov,hcv,P_1,Y_1,V_1,G_1,S_1,L_1,R_1,...,N_8,C_8,T_8,A_8,D_8,K_8,F_8,Q_8,H_8,W_8
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,41,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,41,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
X = df.drop(columns=['virus_id', 'sars_cov', 'hcv'])

In [14]:
y = df['virus_id'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.036298455937344654
KNN:  0.639015987239377
Polynomial SVM degree 2:  0.7272408976663131
Random Forest:  0.7347085062618885
Decision Tree: 0.6846680586915144


In [15]:
y = df['sars_cov'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.574617378434227
KNN:  0.9644713225392122
Polynomial SVM degree 2:  0.9768659609906972
Random Forest:  0.9800223239759976
Decision Tree: 0.9702835302058425


In [16]:
y = df['hcv'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.7550715680406983
KNN:  0.94129883684704
Polynomial SVM degree 2:  0.9549635794009965
Random Forest:  0.9719003955235849
Decision Tree: 0.9502676149984156


In [17]:
y = df['sars_cov'].values.ravel()
replaces = []
replaces.append(['<= 0.50', '== False'])
replaces.append(['>  0.50', '== True'])
build_and_print_decision_tree(X, y, replaces)

Decision Tree: 0.933022854878149
|--- N_1 == False
|   |--- N_2 == False
|   |   |--- N_5 == False
|   |   |   |--- N_4 == False
|   |   |   |   |--- N_3 == False
|   |   |   |   |   |--- Q_2 == False
|   |   |   |   |   |   |--- D_5 == False
|   |   |   |   |   |   |   |--- D_4 == False
|   |   |   |   |   |   |   |   |--- No SARS-CoV inhibition observed
|   |   |   |   |   |   |   |--- D_4 == True
|   |   |   |   |   |   |   |   |--- No SARS-CoV inhibition observed
|   |   |   |   |   |   |--- D_5 == True
|   |   |   |   |   |   |   |--- R_6 == False
|   |   |   |   |   |   |   |   |--- No SARS-CoV inhibition observed
|   |   |   |   |   |   |   |--- R_6 == True
|   |   |   |   |   |   |   |   |--- Likely SARS-CoV inhibition observed
|   |   |   |   |   |--- Q_2 == True
|   |   |   |   |   |   |--- I_1 == False
|   |   |   |   |   |   |   |--- Y_7 == False
|   |   |   |   |   |   |   |   |--- No SARS-CoV inhibition observed
|   |   |   |   |   |   |   |--- Y_7 == True
|   |   |   |  

## 2-Grams 

In [18]:
def build_2_gram_subsequence_dataframe(data, size):
    df, virus_encoder = build_basic_dataframe(data, size)
    symbols, _ = extract_aminoacid_symbols(data)
    encoded_2_gram = []
    for index, x in df.iterrows():
        dictionary = {
            'virus_id': x['virus_id'],
            'sars_cov': x['sars_cov'],
            'hcv': x['hcv']
        }
        for s in symbols:
            if s != '0':
                dictionary[s] = 0
        for x_symbol in x['subsequence']:
            for s in symbols:
                if x_symbol == s and s != '0':
                    dictionary[s] = dictionary[s]+1
                    break
        encoded_2_gram.append(dictionary)
    return pd.DataFrame(encoded_2_gram), virus_encoder

In [19]:
max_sequence_length = data['Sequence'].str.len().max()

### Subsequências de tamanho 8

In [20]:
df, virus_encoder = build_2_gram_subsequence_dataframe(data, 8)
df.head()

,virus_id,sars_cov,hcv,P,Y,V,G,S,L,R,...,N,C,T,A,D,K,F,Q,H,W
0,1,0,0,1,2,1,2,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,2,1,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,1,2,1,1,2,...,0,0,0,0,0,0,0,0,0,0
3,41,0,0,0,1,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,0
4,41,0,0,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [21]:
X = df.drop(columns=['virus_id', 'sars_cov', 'hcv'])

In [22]:
y = df['virus_id'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.03726074943463812
KNN:  0.8153509867932531
Polynomial SVM degree 2:  0.7221212342185556
Random Forest:  0.8655453100234386
Decision Tree: 0.8046113935573794


In [23]:
y = df['hcv'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.8048814096208007
KNN:  0.9720160616427493
Polynomial SVM degree 2:  0.9196274466037233
Random Forest:  0.9779052812462299
Decision Tree: 0.9698602461595558


In [24]:
y = df['sars_cov'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.7111895409608351
KNN:  0.9828321925310279
Polynomial SVM degree 2:  0.9454561475119483
Random Forest:  0.985372771270694
Decision Tree: 0.9809460373979213


### Sequências completas

In [25]:
df, virus_encoder = build_2_gram_subsequence_dataframe(data, max_sequence_length)
df.head()

,virus_id,sars_cov,hcv,P,Y,V,G,S,L,R,...,N,C,T,A,D,K,F,Q,H,W
0,1,0,0,1,2,1,2,1,1,2,...,0,0,0,0,0,0,0,0,0,0
1,41,0,0,0,1,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,0
2,32,0,0,0,1,2,1,2,1,1,...,1,1,1,2,1,0,0,0,0,0
3,32,0,0,0,1,2,1,2,1,0,...,1,0,1,2,1,0,0,0,0,0
4,32,0,0,0,1,2,1,0,1,0,...,1,0,0,2,1,0,0,0,0,0


In [26]:
X = df.drop(columns=['virus_id', 'sars_cov', 'hcv'])

In [27]:
y = df['virus_id'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.2622590575420317
KNN:  0.6590835898650249
Polynomial SVM degree 2:  0.6182666350935354
Random Forest:  0.6780037887757518
Decision Tree: 0.5696968979398531


In [28]:
y = df['hcv'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.7061709685058016
KNN:  0.9218020364669666
Polynomial SVM degree 2:  0.9150059199621122
Random Forest:  0.9378238219275398
Decision Tree: 0.8980203646696662


In [29]:
y = df['sars_cov'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.8552924461283448
KNN:  0.9771820980345727
Polynomial SVM degree 2:  0.9747525455837083
Random Forest:  0.9762088562633199
Decision Tree: 0.9533743784039782


In [30]:
build_and_print_decision_tree(X, y, [])

Decision Tree: 0.9626142552687662
|--- N <= 4.50
|   |--- D <= 4.50
|   |   |--- Q <= 1.50
|   |   |   |--- Y <= 4.50
|   |   |   |   |--- Y <= 2.50
|   |   |   |   |   |--- F <= 1.50
|   |   |   |   |   |   |--- T <= 3.50
|   |   |   |   |   |   |   |--- N <= 3.50
|   |   |   |   |   |   |   |   |--- No SARS-CoV inhibition observed
|   |   |   |   |   |   |   |--- N >  3.50
|   |   |   |   |   |   |   |   |--- No SARS-CoV inhibition observed
|   |   |   |   |   |   |--- T >  3.50
|   |   |   |   |   |   |   |--- M <= 0.50
|   |   |   |   |   |   |   |   |--- No SARS-CoV inhibition observed
|   |   |   |   |   |   |   |--- M >  0.50
|   |   |   |   |   |   |   |   |--- No SARS-CoV inhibition observed
|   |   |   |   |   |--- F >  1.50
|   |   |   |   |   |   |--- N <= 1.50
|   |   |   |   |   |   |   |--- S <= 0.50
|   |   |   |   |   |   |   |   |--- No SARS-CoV inhibition observed
|   |   |   |   |   |   |   |--- S >  0.50
|   |   |   |   |   |   |   |   |--- No SARS-CoV inhibition o

## BLOSUM50 orthonormal

In [31]:
import blosum as bl
def build_blosum50_orthonormal_dataframe(data, size):
    blosum50 = dict(bl.BLOSUM(50))
    df, virus_encoder = build_basic_dataframe(data, size)
    symbols, _ = extract_aminoacid_symbols(data)
    aminoacid_positions = []
    for index, x in df.iterrows():
        subsequence = x['subsequence']
        dictionary = {
            'virus_id': x['virus_id'],
            'sars_cov': x['sars_cov'],
            'hcv': x['hcv']
        }
        for i in range(len(subsequence)):
            if subsequence[i] != '0':
                dictionary[subsequence[i] + '_' + str(i+1)] = blosum50[subsequence[i]+subsequence[i]]
            for s in symbols:
                if s != subsequence[i] and s != '0':
                    dictionary[s + '_' + str(i+1)] = 0
        aminoacid_positions.append(dictionary)
    return pd.DataFrame(aminoacid_positions), virus_encoder

In [32]:
df, virus_encoder = build_blosum50_orthonormal_dataframe(data, 8)
df.head()

,virus_id,sars_cov,hcv,P_1,Y_1,V_1,G_1,S_1,L_1,R_1,...,N_8,C_8,T_8,A_8,D_8,K_8,F_8,Q_8,H_8,W_8
0,1,0,0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,41,0,0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,41,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
X = df.drop(columns=['virus_id', 'sars_cov', 'hcv'])

In [34]:
y = df['virus_id'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.03679887049159286
KNN:  0.637129728356548
Polynomial SVM degree 2:  0.7217749176449525
Random Forest:  0.7347085062618885
Decision Tree: 0.6846680586915144


In [35]:
y = df['hcv'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.7550715680406983
KNN:  0.9320603390600215
Polynomial SVM degree 2:  0.956118298990693
Random Forest:  0.9719003955235849
Decision Tree: 0.9502676149984156


In [ ]:
y = df['sars_cov'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.574617378434227
KNN:  0.9657415896769619
